In [19]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",  # 삭제가능하다. default: history => 모든 key를 history 로 변경하면됨.
    return_messages=True,  ## 대화형 기반 respnose
)
# MessagesPlaceholder: 누가 보냈는지 알 수 없는, 예측하기 어려운 메시지와 제한 없는 양의 메시지를 가져온다.(시스템인지, AI인지, 사람인지 알수없음)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 도움이 되는 인공지능으로 사람과 대화하고 있습니다."),
        MessagesPlaceholder(variable_name="chat_history"),  # memory class 로 대체된다.
        ("human", "{question}"),
    ]
)

In [28]:
## 수동으로 memory 등록
chain = prompt | llm


def chain_invoke(question):
    result = chain.invoke(
        {
            "chat_history": memory.load_memory_variables({})["chat_history"],
            "question": question,
        }
    )
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [33]:
# RunnablePassthrough를 이용하여 메모리자동 입력
def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm


def chain_invoke(question):
    result = chain.invoke(
        {
            "question": question,
        }
    )
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [34]:
chain_invoke("나의 이름은 앤더슨 입니다.")

content='앤더슨님, 반갑습니다! 서울에서 좋아하는 장소나 취미가 있나요? 함께 이야기를 나눠보세요.'


In [35]:
chain_invoke(question="나는 서울에 살고 있습니다.")

content='서울에 사시다니 멋지네요! 서울의 어떤 지역에 거주하시나요? 어떤 장소를 자주 가시나요? 혹은 취미가 있다면 어떤 활동을 즐기시나요? 함께 대화를 나눠보세요.'


In [36]:
chain_invoke(question="나는 김치를 좋아합니다.")

content='김치를 좋아하시는군요! 김치는 한국의 전통 음식 중에서도 매우 인기 있는 음식이죠. 서울에는 다양한 종류의 김치를 맛볼 수 있는 장소들이 많이 있습니다. 예를 들어, 광장시장이나 남대문 시장에서 신선한 김치를 구입하거나, 전통적인 한식당에서 김치찌개나 김치볶음밥을 즐기실 수 있어요. 서울에는 맛있는 김치 요리를 즐길 수 있는 다양한 식당들이 많이 있습니다. 앤더슨님은 서울에서 어떤 장소에서 김치를 가장 좋아하시나요? 혹은 다른 취향 있는 음식이 있나요? 함께 이야기 나눠보세요.'


In [37]:
chain_invoke(question="나는 이름은 무엇입니까?")

content='당신의 이름은 앤더슨입니다. 어떻게 도와드릴까요?'
